    
<h1 style="text-align: center; color: purple;" markdown="1">Econ 220: Python Assignment 4
</h1>

<h2 style="text-align: center; color: #012169" markdown="1">Wooldridge C4.11 (Modified) </h2>



## Package setup

In [1]:
# import necesarry packages
import wooldridge as woo
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms # BP test
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

Use the data in HTV to answer this question. 

## 1. Estimate the regression model

$$ educ = \beta_0 + \beta_1 *motheduc + \beta_2 *fatheduc + \beta_3 *abil + \beta_4 *abil^2 + u$$
by OLS take a look at the results in the usual form but report the results using stargazer. 

## 2.a Test the null hypothesis that educ is linearly related to abil against the alternative that the relationship is quadratic. Show the tstat and the pvalue associated with that test. Do you reject the null hypothesis? what does that mean? EXPLAIN

## 2.b Test the null hypothesis that the effect of mothereduc is bigger 0.3. 

In [2]:
# to include the term ability squared you can create a separate variable 
# or even easier use the I(function) in the ols command to add the term 

htv = woo.dataWoo('htv')
res = smf.ols(formula='educ ~ motheduc + fatheduc + abil + I(abil ** 2)', data = htv).fit()
res.summary()
st = Stargazer([res])
st.rename_covariates({"I(abil ** 2)": "abil2"})
st.covariate_order(['motheduc', 'fatheduc', 'abil', 'I(abil ** 2)', 'Intercept'])
HTML(st.render_html())

In [3]:
# manually confirm the formulas, i.e. extract coefficients and SE:
b = res.params
se = res.bse

print("T-test for linear vs quadratic relation on ability")
# reproduce t statistic:
tstat = b / se # hypothesized value is zero
print(f'tstat: \n {tstat[4]} \n')
df = res.nobs - 4 - 1
# reproduce p value:
pval = 2 * stats.t.cdf(-abs(tstat), df)
# use the np.around() function to round array 
print(f'pval: \n{np.around(pval[4], 4)}\n')

print("T-test for mother educ > 0.3")
# reproduce t statistic:
tstat = (b - 0.3) / se # hypothesized value is 0.3
print(f'tstat: \n {tstat[1]} \n')
df = res.nobs - 4 - 1
# reproduce p value:
pval = stats.t.cdf(-abs(tstat), df)
# use the np.around() function to round array 
print(f'pval: \n{np.around(pval[1], 4)}\n')

T-test for linear vs quadratic relation on ability
tstat: 
 6.093400049308973 

pval: 
0.0

T-test for mother educ > 0.3
tstat: 
 -3.9107002398209647 

pval: 
0.0



## 2.a: The value of beta4 is equal to 0.051, the t test is 6.093, and the p value is close to zero, much less than the significance level. We will reject the null hypothesis that beta4 is zero. Therefore, beta4 is statistically significant,, and we reject that educ is linearly related to abil. Therefore, educ is quadratically related to abil. 

## 2.b: The value of beta one is 0.190, the t test is -3.91, and the p value is also close to zero, much less than the significance level. Therefore, we reject the null hupothesis that effect of motheduc is bigger than 0.3. We can conclude that the effect of motheduc is not bigger than 0.3.


<br>

## 3.  Using the equation in part (2), test $H_0: \beta_1=\beta_2$ against a two-sided alternative. What is the p-value of the test? 

Remember this requires for creating a new regression with a $\theta_1=\beta_1-\beta_2$ as shown in your book and then test for $H_0: \theta_1=0$

 Let's change the regression to create $\theta_1=\beta_1-\beta_2$ 

Add and subrstact $\beta_2 motheduc$ and create a variable $parentedu=motheduc+fatheduc$ see below: 

$$ educ = \beta_0 + \beta_1 motheduc - \beta_2 motheduc + \beta_2 motheduc+ \beta_2 fatheduc + \beta_3 abil + \beta_4 abil^2 + u$$

$$ educ = \beta_0 + (\beta_1 - \beta_2)   motheduc + \beta_2  (motheduc+fatheduc) + \beta_3 abil + \beta_4 abil^2 + u$$
$$ educ = \beta_0 + \theta_1   motheduc + \beta_2  (parentedu) + \beta_3 abil + \beta_4 abil^2 + u$$

By testing the null hypothesis that $H_0:\theta_1=0$ with $alpha=0.05$ we are testing $H_0: \beta_1=\beta_2$ So we just run the regression that has $\theta_1$ as a regressor and look at the t-test for $\theta_1$

In [4]:
#critival Values for alpha=5% and 1%
print("critical values for alpha 5% and 1% 2 tails")
cv_t = [np.around(stats.t.ppf(q=1-.05/2, df=1225.0), 8), np.around(stats.t.ppf(q=1-.01/2, df=1225), 8)]



print(f'cv_t: {cv_t}\n')

# CV for alpha=5% and 1% using the normal approximation:
cv_n = [np.around(stats.norm.ppf(q=1-.05/2), 8), np.around(stats.norm.ppf(q=1-.01/2), 8)]


print(f'cv_n: {cv_n}\n')

#qt(1-alpha_2, 1225)

# create parenteduc
htv['parenteduc'] = htv['motheduc'] + htv['fatheduc']
# regression with theta1
res1 = smf.ols(formula = 'educ ~ motheduc + parenteduc + abil + I(abil ** 2)', data = htv).fit()

st=Stargazer([res1])
st.rename_covariates({"I(abil ** 2)": "abil2"})
st.covariate_order(['motheduc' , 'parenteduc' , 'abil', 'I(abil ** 2)', 'Intercept' ])
HTML(st.render_html())


critical values for alpha 5% and 1% 2 tails
cv_t: [1.96190241, 2.57984871]

cv_n: [1.95996398, 2.5758293]




*** 
#### Complete the empty spaces

The value of $\theta_1$ is equal to 0.081 with a t-stat of 1.936 and a p-value of 0.0531 this means that we fail to reject the null hypothesis that  $H_0:\theta_1=0$ which means that $\beta_1$ equals to $\beta_2$ therefore the level of education of mother's and father's has the same  magnitute. 

***
<br>

## 4. 	Add the two college tuition variables to the regression from part (2) and determine whether they are jointly statistically significant. 
First, do the F-test step-by-step

In [5]:
## F test step by step
n = htv.shape[0]
# Unrestricted OLS regression:  
r2_ur = smf.ols(formula = 'educ ~ motheduc + fatheduc + abil + I(abil ** 2) + tuit17 + tuit18', data = htv).fit()
r2_ur = r2_ur.rsquared

print(f'r2_ur: {r2_ur}\n')

# Restricted OLS regression:
r2_r = smf.ols(formula = 'educ ~ motheduc + fatheduc + abil + I(abil ** 2)', data = htv).fit()
r2_r = r2_r.rsquared




print(f'r2_r: {r2_r}\n')

# F statistic:
fstat = (r2_ur - r2_r)/(1-r2_ur)* (n-7)/ 2



print(f'fstat: {fstat}\n')

# CV for alpha=1% using the F distribution with 3 and 1223 d.f. :
cv = stats.f.ppf(1-0.01, 3, 1223)

print(f'cv: {cv}\n')

fpval = 1 - stats.f.cdf(fstat, 2, 1223)
print(f'fpval: {round(fpval,4)}\n')

r2_ur: 0.44511171101427116

r2_r: 0.4443500858908753

fstat: 0.8393288742998369

cv: 3.797658185020901

fpval: 0.4322




#### Then use any of the other methods


In [6]:
# F test using compare_f_test
r2_ur = smf.ols(formula = 'educ ~ motheduc + fatheduc + abil + I(abil ** 2) + tuit17 + tuit18', data = htv).fit()
r2_r = smf.ols(formula = 'educ ~ motheduc + fatheduc + abil + I(abil ** 2)', data = htv).fit()
r2_ur.compare_f_test(r2_r)





(0.8393288742998524, 0.4322490378049605, 2.0)

<br> 
This shows that in this case we fail to the null hypothesis**, that the coefficients are jointly zero. 

## 5.  Use function `.conf_int()` to find the confidence intervals of all the parameters in the unsrestricted model from (4) What do you conclude? EXPLAIN this results in the light of the significance of your coeficients

In [7]:
#find confidence intervals

r2_ur.conf_int()



,0,1
Intercept,7.468242,8.695487
motheduc,0.137605,0.248180
fatheduc,0.069954,0.146931
abil,0.339500,0.458585
I(abil ** 2),0.034246,0.066863
tuit17,-0.106866,0.138384
tuit18,-0.124810,0.124931


## Explaination:

## For motheduc, fatheduc, abil, and abil^2, the 95% confidence interval does not contain 0, meaning that we are 95% certain that true value of these coefficients are not 0. Therefore, these coefficients are statistically significant. 

## However, for coefficients tuit17 & tuit18, the 95% confidence interval does  contain 0, meaning that we are 95% certain that true value of these coefficients are 0. Therefore, these coefficients are likely not statistically significant.

## Therefore, we can conclude that at 95% confidence level, tuit17 and tuit18 are not statistically significant and should not be added to the regression model. 

In [8]:
!jupyter nbconvert --to html Econ320Lab_Assign4Inference_empty.ipynb

[NbConvertApp] Converting notebook Econ320Lab_Assign4Inference_empty.ipynb to html
[NbConvertApp] Writing 598322 bytes to Econ320Lab_Assign4Inference_empty.html


&nbsp;
<hr />
<p style="font-family:palatino; text-align: center;font-size: 15px">ECON220 Python Programming Laboratory</a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px">Professor <em> Paloma Lopez de mesa Moyano</em></a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px"><span style="color: #6666FF;"><em>paloma.moyano@emory.edu</em></span></p>

<p style="font-family:palatino; text-align: center;font-size: 15px">Department of Economics</a></p>
<p style="font-family:palatino; text-align: center; color: #012169;font-size: 15px">Emory University</a></p>

&nbsp;